<a href="https://colab.research.google.com/github/MAN1108/github-test/blob/main/finetunning%20gpt%203.5%20with%20metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
!pip install -U openai


In [95]:
import pandas as pd
df = pd.read_csv("/content/sample_data/train.csv")
df.head(5)

,question,answer,text
0,What is the name of the place where students b...,Diagon Alley,<s>[INST] <<SYS>> Answer as briefly as you can...
1,What did Wendelin do to make sure she wasn't r...,Wore disguises.,<s>[INST] <<SYS>> Answer as briefly as you can...
2,What is the name of the spell used to disarm a...,Expelliarmus,<s>[INST] <<SYS>> Answer as briefly as you can...
3,What impact does Harry’s friendship with Hermi...,It helps him see the world from different pers...,<s>[INST] <<SYS>> Answer as briefly as you can...
4,What is the name of the magical object that is...,Hermione's Bag,<s>[INST] <<SYS>> Answer as briefly as you can...


In [106]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = f'"{ row["answer"]}"'

        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['question']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data

dataset = pd.read_csv('/content/sample_data/train_1.csv')
converted_data = convert_to_gpt35_format(dataset)
converted_data[0]['messages']

[{'role': 'user',
  'content': 'What is the name of the place where students buy their school supplies?'},
 {'role': 'assistant', 'content': '"Diagon Alley"'}]

In [107]:

import json
json.loads(converted_data[0]['messages'][-1]['content'])

'Diagon Alley'

In [108]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
      random_state=42  # for reproducibility
)

In [109]:
type(train_data[0])

dict

In [114]:
def write_to_jsonl(data, file_path):
    with open(file_path, "w") as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')


training_file_name = "train_1.jsonl"
validation_file_name = "val_1.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [115]:

from openai import OpenAI
client = OpenAI(api_key="sk-proj-G8VutRypXIU9UIEeLoYMT3BlbkFJLy5kDruo7EYTJ84sQjFv")

In [116]:
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)

Training file id: file-OMXb3F9OmEoGnlqIrjHCCoXM
Validation file id: file-Cx8VBjy348iACYf0k0oOIFL1


In [117]:

suffix_name = "Harry_porter_1"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response

FineTuningJob(id='ftjob-8RmoQl8wQzZNzE3GkftlXrpy', created_at=1714630372, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-HZKLAzY4JCgQsHV6XT6RYrNx', result_files=[], seed=1681322092, status='validating_files', trained_tokens=None, training_file='file-OMXb3F9OmEoGnlqIrjHCCoXM', validation_file='file-Cx8VBjy348iACYf0k0oOIFL1', estimated_finish=None, integrations=[], user_provided_suffix='Harry_porter_1')

In [ ]:

client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-jrUvkEQvnm5U4RtmxrNv5M5A', created_at=1712671278, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=8), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-HZKLAzY4JCgQsHV6XT6RYrNx', result_files=[], status='validating_files', trained_tokens=None, training_file='file-gx9y47vilo5zjgWTPbQO9fjS', validation_file='file-XJdoJn2QyjyJJdsH1wnUeNtb', user_provided_suffix='Harry_porter', seed=497932801, integrations=[]), FineTuningJob(id='ftjob-TOAYhvaTjNFeO1W2kChmstVM', created_at=1712669029, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=8), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-HZKLAzY4JCgQsHV6XT6RYrNx', 

In [170]:

response = client.fine_tuning.jobs.retrieve("ftjob-Toh1WzLMPrlvxT2fZl3krdoI")
response

FineTuningJob(id='ftjob-Toh1WzLMPrlvxT2fZl3krdoI', created_at=1714590937, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:question-answering:9KAUxe6O', finished_at=1714593941, hyperparameters=Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-HZKLAzY4JCgQsHV6XT6RYrNx', result_files=['file-qvp96ojUeNImpgFJjlDyLMjw'], seed=91797584, status='succeeded', trained_tokens=157047, training_file='file-VJhx0fw4ZBo6RoAoITxSkCj8', validation_file='file-iwMEh0N1IQyi8CLXoWcSp5ji', estimated_finish=None, integrations=[], user_provided_suffix='Question_Answering')

In [171]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)


Fine-tuned model id: ft:gpt-3.5-turbo-0125:personal:question-answering:9KAUxe6O


In [172]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def format_test(row):

    formatted_message = [
        {
            "role": "user",
            "content": row['question']
        }
    ]
    return formatted_message


def predict(test_messages, fine_tuned_model_id):

    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )

    return response.choices[0].message.content


In [175]:
def store_predictions(test_df, fine_tuned_model_id):

    print("fine_tuned_model_id",fine_tuned_model_id)
    test_df['Prediction'] = None

    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result


    test_df.to_csv("predictions_new_qA.csv")


In [176]:

test_df = pd.read_csv("/content/sample_data/test.csv")
store_predictions(test_df, fine_tuned_model_id)

fine_tuned_model_id ft:gpt-3.5-turbo-0125:personal:question-answering:9KAUxe6O


In [177]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

ground_truth = test_df['answer'].tolist()
predictions = test_df['Prediction'].tolist()


def calculate_metrics(ground_truth, predictions):
  """
  Calculates and prints various evaluation metrics for question answering.

  Args:
      ground_truth: A list of strings containing the correct answers.
      predictions:  A list of strings containing the model's predicted answers.

  Returns:
      A dictionary containing the calculated metrics (accuracy, precision, recall, F1-score).
  """
  metrics = {}

  # Ensure equal list lengths (handle cases with missing predictions)
  ground_truth = ground_truth[:len(predictions)]
  predictions = predictions[:len(ground_truth)]
  predictions = [pred if pred is not None else "I'm not sure" for pred in predictions]
  # Calculate metrics
  metrics["accuracy"] = accuracy_score(ground_truth, predictions)
  metrics["precision"] = precision_score(ground_truth, predictions, average='weighted')  # Weighted precision for imbalanced datasets
  metrics["recall"] = recall_score(ground_truth, predictions, average='weighted')  # Weighted recall for imbalanced datasets
  metrics["f1"] = f1_score(ground_truth, predictions, average='weighted')  # Weighted F1-score for imbalanced datasets

  # Print Metric Results
  print("Evaluation Metrics:")
  for metric_name, value in metrics.items():
    print(f"{metric_name}: {value:.4f}")

  return metrics

# ... (Your code for generating ground_truth and predictions) ...

metrics = calculate_metrics(ground_truth, predictions)

Evaluation Metrics:
accuracy: 0.5874
precision: 0.5796
recall: 0.5874
f1: 0.5753


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
